# Data Analysis
- Based on the Data the most popular and profitable item is "Oathbreaker, Last Hope of the Breaking Storm".
- Although Nirvana is in 4th place of popularity but it is the second most profitable item.
-

In [17]:
import pandas as pd
import numpy as np

file_to_load = "/Users/hessari/Desktop/Instruction/HeroesOfPymoli/Resources/purchase_data.csv"

In [18]:
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [19]:
Player_Count = purchase_data["SN"].nunique()
Player_Count_Show = pd.DataFrame({"Player_Count": [Player_Count]})
Player_Count_Show

,Player_Count
0,576


In [20]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

In [21]:
uniqueitems =purchase_data['Item ID'].nunique()
avgprice = (purchase_data['Price'].sum()/purchase_data['Price'].count())
totalpurchases = purchase_data['Price'].count()
totalrevenue = purchase_data["Price"].sum()

total_analysis_df = pd.DataFrame({"Number of Unique Items": [uniqueitems], 
                                  "Average Purchase Price": [avgprice],
                                  "Number of Purchases": [totalpurchases], 
                                  "Total Revenue": [totalrevenue]}, columns= ["Number of Unique Items", "Average Purchase Price",
                                                                              "Number of Purchases", "Total Revenue"])

In [22]:
total_analysis_df.style.format({"Average Purchase Price": "${:.2f}", "Total Revenue": "${:.2f}"})

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [23]:
duplicate = purchase_data.drop_duplicates(subset='SN', keep="first")
TotalGenderCount = duplicate["Gender"].count()
MaleGender = duplicate["Gender"].value_counts()['Male']
FemaleGender = duplicate["Gender"].value_counts()['Female']
NonGender = TotalGenderCount - MaleGender - FemaleGender
MalePercent = (MaleGender / TotalGenderCount) * 100
FemalePercent = (FemaleGender / TotalGenderCount) * 100
NonPercent = (NonGender / TotalGenderCount) * 100

In [24]:
Gender_Demographic = pd.DataFrame({"Gender": ['Male', 'Female', 'Other/Non-Disclosed'],
                                   "Total Count": [MaleGender, FemaleGender, NonGender],
                                   "Percentage of Players": [MalePercent, FemalePercent, NonPercent]})

Gender_Demographic_Final = Gender_Demographic.set_index("Gender")
Gender_Demographic_Final.style.format({"Percentage of Players": "{:.2f}%"})

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [25]:
Grouped_df = purchase_data.groupby(["Gender"])
PurchCount = Grouped_df["SN"].count()
PurchPrice = Grouped_df["Price"].mean()
PurchValue = Grouped_df["Price"].sum()
Duplicate = purchase_data.drop_duplicates(subset='SN', keep="first")
Grouped_Dup = duplicate.groupby(["Gender"])
AveragePurchPP = (Grouped_df["Price"].sum() / Grouped_Dup["SN"].count())

In [26]:
Purchase_Analys_Gender = pd.DataFrame({"Purchase Count": PurchCount,
                              "Average Purchase Price": PurchPrice,
                              "Total Purchase Value": PurchValue,
                              "Average Purchase PP": AveragePurchPP})
Purchase_Analys_Gender.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:.2f}",
                                    "Average Purchase PP": "${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase PP
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [27]:
Bin = [0,10,15,20,25,30,35,40,200]
AgeDemo = ['<10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', '40+']

In [28]:
Binner_df = purchase_data.copy()
Binner_df["Age Groups"] = pd.cut(Binner_df["Age"], Bin, labels=AgeDemo)
Group_Bin = Binner_df.groupby(["Age Groups"])
BinnerCount = Group_Bin["SN"].count()
CountTotal = purchase_data["SN"].count()
Percentage = (BinnerCount / CountTotal) * 100
Age_Perc = pd.DataFrame({"Total Count": BinnerCount,
                         "Percentage of Players": Percentage})
Age_Perc["Percentage of Players"] = Age_Perc["Percentage of Players"].map("{:.2f}%".format)
Age_Perc.head(10)

,Total Count,Percentage of Players
Age Groups,,
<10,32,4.10%
10 - 14,54,6.92%
15 - 19,200,25.64%
20 - 24,325,41.67%
25 - 29,77,9.87%
30 - 34,52,6.67%
35 - 39,33,4.23%
40+,7,0.90%


In [29]:
Top_Spenders = purchase_data['Item ID'].groupby(purchase_data['SN']).count() 
Top_Spenders= pd.DataFrame(data=Top_Spenders)
Top_Spenders.columns = ['Purchase Count']
Top_Spenders['Average Purchase Price'] = purchase_data['Price'].groupby(purchase_data['SN']).mean()
Top_Spenders['Total Purchase Value'] = purchase_data['Price'].groupby(purchase_data['SN']).sum()
Top_Spenders.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)
Top_Spenders['Average Purchase Price'] = Top_Spenders['Average Purchase Price'].map('${:,.2f}'.format)
Top_Spenders['Total Purchase Value'] = Top_Spenders['Total Purchase Value'].map('${:,.2f}'.format)

Top_Spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [30]:
Most_Pop_Items = purchase_data.groupby(['Item ID', 'Item Name','Price'])['Price'].agg(['count','sum'])
Most_Pop_Items.columns = ['Purchase Count', 'Total Purchase Value']
Most_Pop_Items.reset_index(inplace=True)
Most_Pop_Items.set_index(['Item ID','Item Name'] ,inplace=True)

Most_Pop_Items = Most_Pop_Items[['Purchase Count', 'Price', 'Total Purchase Value']]

Most_Pop_Items_One = Most_Pop_Items.sort_values(by='Purchase Count', ascending=False)
 
Most_Pop_Items_One['Price'] = Most_Pop_Items_One['Price'].map('${:,.2f}'.format)
Most_Pop_Items_One['Total Purchase Value'] = Most_Pop_Items_One['Total Purchase Value'].map('${:,.2f}'.format)

Most_Pop_Items_One.head(10)

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16
60,Wolf,8,$3.54,$28.32


In [31]:
Most_Profitable_Items = Most_Pop_Items.sort_values(by='Total Purchase Value', ascending=False)
Most_Profitable_Items['Price'] = Most_Profitable_Items['Price'].map('${:,.2f}'.format)
Most_Profitable_Items['Total Purchase Value'] = Most_Profitable_Items['Total Purchase Value'].map('${:,.2f}'.format)
Most_Profitable_Items.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
